# How to Configure k-Fold Cross-Validation

by Jason Brownlee on July 31, 2020.[Here](https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/) in [Python Machine Learning](https://machinelearningmastery.com/category/python-machine-learning/)

The k-fold cross-validation procedure is a standard method for estimating the performance of a machine learning algorithm on a dataset.

A common value for k is 10, although `how do we know that this configuration is appropriate for our dataset and our algorithms`?

__One approach__ is to explore the effect of different k values on the estimate of model performance and compare this to an ideal test condition. This can help to choose an appropriate value for k.

__Once a k-value is chosen__, it can be used to evaluate a suite of different algorithms on the dataset and the distribution of results can be compared to an evaluation of the same algorithms using an ideal test condition to see if they are highly correlated or not. If correlated, it confirms the chosen configuration is a robust approximation for the ideal test condition.

After completing this tutorial, you will know:

- How to evaluate a machine learning algorithm using k-fold cross-validation on a dataset.
- How to perform a sensitivity analysis of k-values for k-fold cross-validation.
- How to calculate the correlation between a cross-validation test harness and an ideal test condition.

## Tutorial Overview
This tutorial is divided into three parts; they are:

1. k-Fold Cross-Validation
2. Sensitivity Analysis for k
3. Correlation of Test Harness With Target